# Imports

In [1]:
config = {
    'dhdt': {
        'depth': 3,
        'learning_rate': 0.01,#1e-3,
        
        'initializer': 'GlorotUniform', #GlorotUniform
        
        'loss': 'binary_crossentropy',#'mae',
        'optimizer': 'adam',        
        
        'beta_1': 10,
        'beta_2': 50,
        
        'squeeze_factor': 1,
        
        'batch_size': 512,
        'epochs': 1_000,
        'early_stopping_epochs': 50,
    },
    
    
    
    'make_classification': {
        'number_of_variables': 10,
        'n_samples': 5_000,
        'num_eval': 50,
    },

    'computation': {
        'random_seed': 42,
        'trials': 5,
        'n_jobs': 50,
        'verbosity': 0,
    },
}



In [2]:
import numpy as np

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder, OrdinalEncoder

from livelossplot import PlotLosses

import os
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt

from IPython.display import Image
from IPython.display import display, clear_output

import pandas as pd

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = '' #'true'

import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import logging

import tensorflow as tf
import tensorflow_addons as tfa

tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(3)

np.seterr(all="ignore")

from keras import backend as K
from keras.utils.generic_utils import get_custom_objects


import seaborn as sns
sns.set_style("darkgrid")

import time
import random

from utilities.utilities import *
from utilities.DHDT import *

from joblib import Parallel, delayed

from itertools import product
from collections.abc import Iterable

from copy import deepcopy

# Evaluation

## make_classification

In [3]:
if False:
    config_test = deepcopy(config)
    config_test['make_classification']['n_samples'] = 10_000
    config_test['dhdt']['epochs'] = 3

    dataset_dict = {}
    model_dict = {}

    scores_dict = {'sklearn': {},
                   'DHDT': {}}

    dataset_dict = get_preprocessed_dataset('make_classification',
                                            random_seed=config_test['computation']['random_seed'],
                                            config=config_test['make_classification'],
                                            verbosity=1)

    model_dict['sklearn'] = DecisionTreeClassifier(max_depth=3, 
                                                   random_state=config_test['computation']['random_seed'])

    model_dict['sklearn'].fit(dataset_dict['X_train'], 
                              dataset_dict['y_train'])



    model_dict['DHDT'] = DHDT(dataset_dict['X_train'].shape[1],

                                depth = config_test['dhdt']['depth'],

                                learning_rate = config_test['dhdt']['learning_rate'],
                                optimizer = config_test['dhdt']['optimizer'],

                                beta_1 = config_test['dhdt']['beta_1'],
                                beta_2 = config_test['dhdt']['beta_2'],

                                squeeze_factor = config_test['dhdt']['squeeze_factor'],

                                loss = config_test['dhdt']['loss'],#'mae',

                                random_seed = config_test['computation']['random_seed'],
                                verbosity = 2)        


    scores_dict['history'] = model_dict['DHDT'].fit(dataset_dict['X_train'], 
                                                  dataset_dict['y_train'], 
                                                  batch_size=config_test['dhdt']['batch_size'], 
                                                  epochs=config_test['dhdt']['epochs'], 
                                                  early_stopping_epochs=config_test['dhdt']['early_stopping_epochs'], 
                                                  valid_data=(dataset_dict['X_valid'], dataset_dict['y_valid']))



    scores_dict['sklearn']['accuracy_test'] = model_dict['sklearn'].score(dataset_dict['X_test'], 
                                                                     dataset_dict['y_test'])


    dataset_dict['y_test_dhdt'] = model_dict['DHDT'].predict(dataset_dict['X_test'])
    scores_dict['DHDT']['accuracy_test'] = accuracy_score(dataset_dict['y_test'], np.round(dataset_dict['y_test_dhdt']))


In [4]:
if True:
    parallel_eval_synthetic = Parallel(n_jobs=config['computation']['n_jobs'], verbose=3, backend='loky') #loky #sequential multiprocessing
    evaluation_results_synthetic = parallel_eval_synthetic(delayed(evaluate_synthetic_parallel)(index = index,
                                                                                                random_seed_data = config['computation']['random_seed']+index,
                                                                                                random_seed_model = config['computation']['random_seed'],#+random_seed_model,
                                                                                                config = config,
                                                                                                verbosity = -1) for index in range(config['make_classification']['num_eval']))

    for i, synthetic_result in enumerate(evaluation_results_synthetic):
        if i == 0:
            model_dict_synthetic = synthetic_result[0]
            scores_dict_synthetic = synthetic_result[1]
            dataset_dict_synthetic = synthetic_result[2]
        else: 
            model_dict_synthetic = mergeDict(model_dict_synthetic, synthetic_result[0])
            scores_dict_synthetic = mergeDict(scores_dict_synthetic, synthetic_result[1])
            dataset_dict_synthetic = mergeDict(dataset_dict_synthetic, synthetic_result[2])


    metrics = ['accuracy_test']
    index = [i for i in range(config['make_classification']['num_eval'])]
    columns = flatten_list([[[approach + ' ' + metric + '_mean', approach + ' ' + metric + '_max', approach + ' ' + metric + '_std'] for metric in metrics] for approach in ['DHDT', 'sklearn']])

    scores_DHDT = [scores_dict_synthetic[i]['DHDT'][metrics[0]] for i in range(config['make_classification']['num_eval'])]

    scores_sklearn = [scores_dict_synthetic[i]['sklearn'][metrics[0]] for i in range(config['make_classification']['num_eval'])]


    scores_DHDT_mean = np.mean(scores_DHDT, axis=1) if config['computation']['trials'] > 1 else scores_DHDT
    scores_sklearn_mean = np.mean(scores_sklearn, axis=1) if config['computation']['trials'] > 1 else scores_sklearn

    scores_DHDT_max = np.max(scores_DHDT, axis=1) if config['computation']['trials'] > 1 else scores_DHDT
    scores_sklearn_max = np.max(scores_sklearn, axis=1) if config['computation']['trials'] > 1 else scores_sklearn

    scores_DHDT_std = np.std(scores_DHDT, axis=1) if config['computation']['trials'] > 1 else np.array([0.0] * config['make_classification']['num_eval'])
    scores_sklearn_std = np.std(scores_sklearn, axis=1) if config['computation']['trials'] > 1 else np.array([0.0] * config['make_classification']['num_eval'])

    results_DHDT = np.vstack([scores_DHDT_mean, scores_DHDT_max, scores_DHDT_std])
    results_sklearn = np.vstack([scores_sklearn_mean, scores_sklearn_max, scores_sklearn_std])


    scores_dataframe_synthetic = pd.DataFrame(data=np.vstack([results_DHDT, results_sklearn]).T, index = index, columns = columns)
    display(scores_dataframe_synthetic)
    display(scores_dataframe_synthetic[scores_dataframe_synthetic.columns[1::3]])
    display(scores_dataframe_synthetic.describe())

[Parallel(n_jobs=50)]: Using backend LokyBackend with 50 concurrent workers.
[Parallel(n_jobs=50)]: Done   2 out of  50 | elapsed:  4.5min remaining: 108.9min
[Parallel(n_jobs=50)]: Done  19 out of  50 | elapsed:  5.1min remaining:  8.4min
[Parallel(n_jobs=50)]: Done  36 out of  50 | elapsed:  5.6min remaining:  2.2min
[Parallel(n_jobs=50)]: Done  50 out of  50 | elapsed:  6.7min finished


,DHDT accuracy_test_mean,DHDT accuracy_test_max,DHDT accuracy_test_std,sklearn accuracy_test_mean,sklearn accuracy_test_max,sklearn accuracy_test_std
0,0.7032,0.754,0.037220,0.7724,0.814,0.0208
1,0.7154,0.817,0.097678,0.8292,0.849,0.0396
2,0.7222,0.771,0.032220,0.8114,0.857,0.0228
3,0.7278,0.748,0.022284,0.7814,0.796,0.0292
4,0.6046,0.733,0.066217,0.6840,0.820,0.0680
5,0.7758,0.802,0.029485,0.8224,0.840,0.0352
6,0.6912,0.721,0.022964,0.7542,0.760,0.0116
7,0.7918,0.851,0.032908,0.8552,0.884,0.0144
8,0.8252,0.868,0.080153,0.8424,0.867,0.0492
9,0.7018,0.796,0.049648,0.7892,0.798,0.0044


,DHDT accuracy_test_max,sklearn accuracy_test_max
0,0.754,0.814
1,0.817,0.849
2,0.771,0.857
3,0.748,0.796
4,0.733,0.820
5,0.802,0.840
6,0.721,0.760
7,0.851,0.884
8,0.868,0.867
9,0.796,0.798


,DHDT accuracy_test_mean,DHDT accuracy_test_max,DHDT accuracy_test_std,sklearn accuracy_test_mean,sklearn accuracy_test_max,sklearn accuracy_test_std
count,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,0.685936,0.749000,0.050177,0.772956,0.804420,0.026064
std,0.052478,0.054334,0.021174,0.052294,0.052834,0.018940
min,0.590000,0.637000,0.022031,0.639400,0.647000,0.000000
25%,0.647600,0.708000,0.033888,0.740050,0.777750,0.012800
50%,0.689600,0.750500,0.047524,0.783500,0.808000,0.023600
75%,0.710800,0.782000,0.060473,0.815450,0.840000,0.035800
max,0.825200,0.868000,0.124049,0.855200,0.895000,0.081200


## Real-World Eval

In [5]:
if False:

    identifier_list = [
                        'Adult',#: 32,
                        'Bank Marketing',#: 32,
                        'Loan Credit',#: 32,

                        'Credit Card',#: 23, 
                        'Car',#: 21,


                        'Absenteeism',#: 15,
                        'Loan House',#: 15,
                        'Cervical Cancer',#: 15,

                        'Heart Disease',#: 13,           

                        'Titanic',#: 10,
                        'Medical Insurance',#: 10,
                        'Wisconsin Breast Cancer Original',#: 10,
                        'Wisconsin Diagnostic Breast Cancer',#: 10,
                        'Wisconsin Prognostic Breast Cancer',#: 10,
                        'Abalone',#: 10,

                        'Habermans Survival',#: 3, 
                      ]

    #identifier_list = ['Habermans Survival']

    parallel_eval_real_world = Parallel(n_jobs=config['computation']['n_jobs'], verbose=3, backend='loky') #loky #sequential multiprocessing
    evaluation_results_real_world = parallel_eval_real_world(delayed(evaluate_real_world_parallel)(identifier_list=identifier_list, 
                                                                                                   random_seed_model=config['computation']['random_seed']+i,
                                                                                                   config = config,
                                                                                                   verbosity = -1) for i in range(config['computation']['trials']))


    for i, real_world_result in enumerate(evaluation_results_real_world):
        if i == 0:
            model_dict_real_world = real_world_result[0]
            scores_dict_real_world = real_world_result[1]
            dataset_dict_real_world = real_world_result[2]
        else: 
            model_dict_real_world = mergeDict(model_dict_real_world, real_world_result[0])
            scores_dict_real_world = mergeDict(scores_dict_real_world, real_world_result[1])
            dataset_dict_real_world = mergeDict(dataset_dict_real_world, real_world_result[2])

    metrics = ['accuracy_test']
    index = identifier_list
    columns = flatten_list([[[approach + ' ' + metric + '_mean', approach + ' ' + metric + '_max', approach + ' ' + metric + '_std'] for metric in metrics] for approach in ['DHDT', 'sklearn']])

    scores_DHDT = [scores_dict_real_world[identifier]['DHDT'][metrics[0]] for identifier in identifier_list]

    scores_sklearn = [scores_dict_real_world[identifier]['sklearn'][metrics[0]] for identifier in identifier_list]

    scores_DHDT_mean = np.mean(scores_DHDT, axis=1) if config['computation']['trials'] > 1 else scores_DHDT
    scores_sklearn_mean = np.mean(scores_sklearn, axis=1) if config['computation']['trials'] > 1 else scores_sklearn

    scores_DHDT_max = np.max(scores_DHDT, axis=1) if config['computation']['trials'] > 1 else scores_DHDT
    scores_sklearn_max = np.max(scores_sklearn, axis=1) if config['computation']['trials'] > 1 else scores_sklearn

    scores_DHDT_std = np.std(scores_DHDT, axis=1) if config['computation']['trials'] > 1 else np.array([0.0] * config['computation']['trials'])
    scores_sklearn_std = np.std(scores_sklearn, axis=1) if config['computation']['trials'] > 1 else np.array([0.0] * config['computation']['trials'])

    results_DHDT = np.vstack([scores_DHDT_mean, scores_DHDT_max, scores_DHDT_std])
    results_sklearn = np.vstack([scores_sklearn_mean, scores_sklearn_max, scores_sklearn_std])


    scores_dataframe_real_world = pd.DataFrame(data=np.vstack([results_DHDT, results_sklearn]).T, index = index, columns = columns)
    display(scores_dataframe_real_world)
    display(scores_dataframe_real_world[scores_dataframe_real_world.columns[1::3]])



In [6]:
if False:
    identifier = identifier_list[0]#"Absenteeism"
    plt.figure(figsize=(15,8))
    image = model_dict_real_world[identifier]['DHDT'].plot(normalizer_list=dataset_dict_real_world[identifier]['normalizer_list'])
    display(image)

    plt.figure(figsize=(15,8))
    plot_tree(model_dict_real_world[identifier]['sklearn'], fontsize=10) 
    plt.show()

# Hyperparameter Optimization

In [7]:
parameter_dict = {
        'depth': [3],
        'learning_rate': [0.05, 0.01, 0.005, 0.001, 0.0005],#[0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001],
        
        'loss': ['binary_crossentropy'],#['binary_crossentropy', 'rmse'],#'mae',
        'optimizer': ['adam', 'sgd'],#['adam', 'sgd'],        
        
        'beta_1': [10, 50],#[10, 50, 100],
        'beta_2': [10, 50],#[10, 50, 100],
        
        'squeeze_factor': [0.2, 0.5, 1, 2, 5],#[0.2, 0.5, 1, 2, 5],    
}

parameter_dict = {
        #'depth': [3],
        #'learning_rate': [0.05, 0.01, 0.005, 0.001, 0.0005],#[0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 0.0001],
        
        'loss': ['binary_crossentropy'],#['binary_crossentropy', 'rmse'],#'mae',
        'optimizer': ['adam', 'sgd'],#['adam', 'sgd'],        
        
        #'beta_1': [10, 50],#[10, 50, 100],
        #'beta_2': [10, 50],#[10, 50, 100],
        
        #'squeeze_factor': [0.2, 0.5, 1, 2, 5],#[0.2, 0.5, 1, 2, 5],    
}

parameter_grid = ParameterGrid(parameter_dict)

In [8]:
if False:
    parallel_hpo_synthetic = Parallel(n_jobs=config['computation']['n_jobs'], verbose=3, backend='loky') #loky #sequential multiprocessing
    evaluation_results_hpo_synthetic = parallel_hpo_synthetic(delayed(evaluate_parameter_setting_synthetic)(parameter_setting) for parameter_setting in parameter_grid)

    sorted_evaluation_results_hpo_synthetic = sorted(evaluation_results_hpo_synthetic, key=lambda x: x[0], reverse=True)
    print(sorted_evaluation_results_hpo_synthetic[:5])

In [9]:
if False:
    identifier_list = [
                        'Adult',#: 32,
                        'Bank Marketing',#: 32,
                        'Loan Credit',#: 32,

                        'Credit Card',#: 23, 
                        'Car',#: 21,


                        'Absenteeism',#: 15,
                        'Loan House',#: 15,
                        'Cervical Cancer',#: 15,

                        'Heart Disease',#: 13,           

                        'Titanic',#: 10,
                        'Medical Insurance',#: 10,
                        'Wisconsin Breast Cancer Original',#: 10,
                        'Wisconsin Diagnostic Breast Cancer',#: 10,
                        'Wisconsin Prognostic Breast Cancer',#: 10,
                        'Abalone',#: 10,

                        'Habermans Survival',#: 3, 
                      ]

    sorted_evaluation_results_hpo_real_dict = {}

    for identifier in identifier_list:
        parallel_hpo_real = Parallel(n_jobs=config['computation']['n_jobs'], verbose=3, backend='loky') #loky #sequential multiprocessing
        evaluation_results_hpo_real = parallel_hpo_real(delayed(evaluate_parameter_setting_real_world)(parameter_setting, identifier) for parameter_setting in parameter_grid)

        sorted_evaluation_results_hpo_real = sorted(evaluation_results_hpo_real, key=lambda x: x[0], reverse=True)

        sorted_evaluation_results_hpo_real_dict[identifier] = sorted_evaluation_results_hpo_real

        print(sorted_evaluation_results_hpo_real[:5])
        